## Търсене и извличане на информация. Приложение на дълбоко машинно обучение
## Стоян Михов
## Зимен семестър 2024/2025

In [1]:
import sys
import nltk
from nltk.corpus import PlaintextCorpusReader
import numpy as np
import torch
import random
import math

In [2]:
class progressBar:
    def __init__(self ,barWidth = 50):
        self.barWidth = barWidth
        self.period = None
    def start(self, count):
        self.item=0
        self.period = int(count / self.barWidth)
        sys.stdout.write("["+(" " * self.barWidth)+"]")
        sys.stdout.flush()
    def tick(self):
        if self.period == 0: return
        if self.item>0 and self.item % self.period == 0:
            progress = self.item // self.period
            sys.stdout.write(f"\r[{'-'* progress}{' '*(self.barWidth - progress)}]")
            sys.stdout.flush()
        self.item += 1
    def stop(self):
        sys.stdout.write("\r["+("-" * self.barWidth)+"]\n")

In [3]:
def extractDictionary(corpus, limit=20000):
    pb = progressBar()
    pb.start(len(corpus))
    dictionary = {}
    for doc in corpus:
        pb.tick()
        for w in doc:
            if w not in dictionary: dictionary[w] = 0
        dictionary[w] += 1
    L = sorted([(w,dictionary[w]) for w in dictionary], key = lambda x: x[1] , reverse=True)
    if limit > len(L): limit = len(L)
    words = [ w for w,_ in L[:limit] ] + [unkToken]
    word2ind = { w:i for i,w in enumerate(words)}
    pb.stop()
    return words, word2ind

def splitSentCorpus(fullSentCorpus, testFraction = 0.1):
    random.seed(42)
    random.shuffle(fullSentCorpus)
    testCount = int(len(fullSentCorpus) * testFraction)
    testSentCorpus = fullSentCorpus[:testCount]
    trainSentCorpus = fullSentCorpus[testCount:]
    return testSentCorpus, trainSentCorpus

def extractData(corpus, order, word2ind):
    pb = progressBar()
    pb.start(len(corpus))
    unk = word2ind[unkToken]
    start = word2ind[startToken]

    points = sum(len(s)-1 for s in corpus)
    
    target = np.empty(points, dtype='int32')
    context = np.empty((points,order-1), dtype='int32')
    p = 0
    for doc in corpus:
        #pb.tick()
        for wi in range(1,len(doc)):
            i = word2ind.get(doc[wi], unk)
            target[p] = i
            for k in range(1,order):
                if wi-k < 0:
                    j = start
                else:
                    j = word2ind.get(doc[wi-k], unk)
                context[p,k-1] = j
            p += 1
    pb.stop()
    return target, context


#   Зареждане на корпуса

In [4]:
corpus_root = '/home/cdnomlqko/D/Boby/programming/Standalone/nlp/JOURNALISM.BG/C-MassMedia'
myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')
startToken = '<START>'
endToken = '<END>'
unkToken = '<UNK>'

corpus = [ [startToken] + [w.lower() for w in sent] + [endToken] for sent in myCorpus.sents()]

words, word2ind = extractDictionary(corpus)

testCorpus, trainCorpus  = splitSentCorpus(corpus, testFraction = 0.01)

order = 4
target, context = extractData(trainCorpus, order, word2ind)

emb_size = 50
hid_size = 100

L = len(words)

batchSize = 1000
idx = np.arange(len(target), dtype='int32')
np.random.shuffle(idx)
learning_rate = 1.


<>:2: SyntaxWarning: invalid escape sequence '\.'
<>:2: SyntaxWarning: invalid escape sequence '\.'
/tmp/ipykernel_378504/1007442835.py:2: SyntaxWarning: invalid escape sequence '\.'
  myCorpus = PlaintextCorpusReader(corpus_root, '.*\.txt')


[--------------------------------------------------]
[--------------------------------------------------]


In [5]:
#device = torch.device("cpu")
#device = torch.device("cuda:0")
device = torch.device("cuda:0")
#device = torch.device("vulkan")
#device = torch.device("mps")

# Параметри на модела

In [6]:
E = torch.empty(L, emb_size, device=device, requires_grad = True)
W1 = torch.empty((order-1)*emb_size, hid_size, device=device, requires_grad = True)
b1 = torch.empty(hid_size, device=device, requires_grad = True)
W2 = torch.empty(hid_size, L, device=device, requires_grad = True)
b2 = torch.empty(L, device=device, requires_grad = True)

### моделът (Bengio et al.)

$$
X = concat_i(E_{\mathcal{X}_i})
$$
$$
h = \sigma(W^{(1)}X + b^{(1)})
$$
$$
Y = softmax(W^{(2)}h + b^{(2)})
$$
минимизираме кросентропия

## Стар вариант

In [7]:
torch.nn.init.normal_(E)
torch.nn.init.normal_(W1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(W2)
torch.nn.init.normal_(b2)

for b in range(0,len(idx),batchSize):
    
    batchIdx = idx[b:min(b+batchSize,len(idx))]
    S = len(batchIdx)
    batchTarget = torch.tensor(target[batchIdx], device=device, dtype=torch.long)
    batchContext = context[batchIdx]
    
    X = E[batchContext].view(S,(order-1) * emb_size)
    h = torch.sigmoid(torch.matmul(X,W1) + b1)
    z = torch.matmul(h,W2) + b2
    H = torch.nn.functional.cross_entropy(z,batchTarget)
    
    H.backward()
    
    with torch.no_grad():
        E -= learning_rate * E.grad
        W1 -= learning_rate * W1.grad
        b1 -= learning_rate * b1.grad
        W2 -= learning_rate * W2.grad
        b2 -= learning_rate * b2.grad
        # Manually zero the gradients
        E.grad = None
        W1.grad = None
        b1.grad = None
        W2.grad = None
        b2.grad = None
    
    if b % 10000 == 0:
        print(b, '/', len(idx), H.item())

0 / 9892431 27.12305450439453
10000 / 9892431 20.887889862060547
20000 / 9892431 18.747011184692383
30000 / 9892431 17.221284866333008
40000 / 9892431 16.469011306762695
50000 / 9892431 15.522582054138184
60000 / 9892431 15.997777938842773
70000 / 9892431 15.17389965057373
80000 / 9892431 14.758880615234375
90000 / 9892431 14.41395092010498
100000 / 9892431 13.873919486999512
110000 / 9892431 14.070328712463379
120000 / 9892431 13.055603981018066
130000 / 9892431 12.952430725097656
140000 / 9892431 13.272693634033203
150000 / 9892431 13.138757705688477
160000 / 9892431 12.480018615722656
170000 / 9892431 12.745646476745605
180000 / 9892431 11.787392616271973
190000 / 9892431 11.83434772491455
200000 / 9892431 11.85159969329834
210000 / 9892431 11.080212593078613
220000 / 9892431 11.335232734680176
230000 / 9892431 10.964300155639648
240000 / 9892431 10.641793251037598
250000 / 9892431 10.859771728515625
260000 / 9892431 10.886767387390137
270000 / 9892431 10.767422676086426
280000 / 98

##   Дефиниране на нова функция за афинна трансформация

In [8]:
class AfineFunction(torch.autograd.Function):
    
    # Note that both forward and backward are @staticmethods
    @staticmethod
    # This function implements output = input @ weight + bias
    def forward(ctx, input, weight, bias):
        ctx.save_for_backward(input, weight)
        output = torch.mm(input,weight)
        output += bias.unsqueeze(0)
        return output
    
    # This function gets the gradient for its output
    @staticmethod
    def backward(ctx, grad_output):
        input, weight = ctx.saved_tensors
        
        grad_input = torch.mm(grad_output, weight.t())
        grad_weight = torch.mm(input.t(), grad_output)
        grad_bias = grad_output.sum(0)
        
        return grad_input, grad_weight, grad_bias

##    вариант използващ афинната функция

In [ ]:
torch.nn.init.normal_(E)
torch.nn.init.normal_(W1)
torch.nn.init.normal_(b1)
torch.nn.init.normal_(W2)
torch.nn.init.normal_(b2)

for b in range(0,len(idx),batchSize):
    
    batchIdx = idx[b:min(b+batchSize,len(idx))]
    batchTarget = torch.tensor(target[batchIdx], device=device, dtype=torch.long)
    batchContext = context[batchIdx]
    
    X = E[batchContext].flatten(1,2)
    h = torch.sigmoid(AfineFunction.apply(X,W1,b1))
    z = AfineFunction.apply(h,W2,b2)
    H = torch.nn.functional.cross_entropy(z,batchTarget)
    
    H.backward()
    
    with torch.no_grad():
        E -= learning_rate * E.grad
        W1 -= learning_rate * W1.grad
        b1 -= learning_rate * b1.grad
        W2 -= learning_rate * W2.grad
        b2 -= learning_rate * b2.grad
        # Manually zero the gradients
        E.grad = None
        W1.grad = None
        b1.grad = None
        W2.grad = None
        b2.grad = None
    
    if b % 10000 == 0:
        print(b, '/', len(idx), H.item())


In [8]:
del E
del W1
del W2
del b1
del b2

## Вариант с използване на модул

In [ ]:
class LModel(torch.nn.Module):
    def __init__(self, L, emb_size, hid_size, order):
        super(LModel, self).__init__()
        
        self.E = torch.nn.Parameter(torch.rand(L, emb_size)-0.5)
        self.W1 = torch.nn.Parameter(torch.rand((order-1)*emb_size, hid_size)-0.5)
        self.b1 = torch.nn.Parameter(torch.rand(hid_size)-0.5)
        self.W2 = torch.nn.Parameter(torch.rand(hid_size, L)-0.5)
        self.b2 = torch.nn.Parameter(torch.rand(L)-0.5)
    
    def forward(self, context, target):
        device = next(self.parameters()).device

        targetTensor = torch.tensor(target, dtype=torch.long, device = device)
        
        X = self.E[context].flatten(1,2)
        h = torch.sigmoid(torch.matmul(X,self.W1) + self.b1)
        z = torch.matmul(h,self.W2) + self.b2
        H = torch.nn.functional.cross_entropy(z,targetTensor)
        return H

model = LModel(L, emb_size, hid_size, order).to(device)
for p in model.parameters(): print(p.shape)

for b in range(0,len(idx),batchSize):
    
    batchIdx = idx[b:min(b+batchSize,len(idx))]
    H = model(context[batchIdx],target[batchIdx])
    
    model.zero_grad()
    H.backward()
    
    with torch.no_grad():
        for param in model.parameters():
            param -= learning_rate * param.grad

    if b % 10000 == 0:
        print(b, '/', len(idx), H.item())


In [ ]:
del model

##   Вариант с модул, в който са вложени други модули

In [9]:
class LModel(torch.nn.Module):
    def __init__(self, L, emb_size, hid_size, order):
        super(LModel, self).__init__()
        
        self.embedding = torch.nn.Embedding(L, emb_size)
        self.layer1 = torch.nn.Linear((order-1) * emb_size, hid_size)
        self.layer2 = torch.nn.Linear(hid_size, L)
    
    def forward(self, context, target):
        device = next(self.parameters()).device
        targetTensor = torch.tensor(target, dtype=torch.long, device = device)
        contextTensor = torch.tensor(context, dtype=torch.long, device = device)

        X = self.embedding(contextTensor).flatten(1,2)
        h = torch.sigmoid(self.layer1(X))
        z = self.layer2(h)
        return torch.nn.functional.cross_entropy(z,targetTensor)

model = LModel(L, emb_size, hid_size, order).to(device)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate)
#optimizer = torch.optim.SGD(model.parameters(), lr=learning_rate, momentum=0.9)
optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.000001)

for b in range(0,len(idx),batchSize):
    
    batchIdx = idx[b:min(b+batchSize,len(idx))]
    H = model(context[batchIdx],target[batchIdx])
    
    optimizer.zero_grad()
    H.backward()
    optimizer.step()

    if b % 10000 == 0:
        print(b, '/', len(idx), H.item())

0 / 9892431 9.972273826599121
10000 / 9892431 6.4774932861328125
20000 / 9892431 6.258452415466309
30000 / 9892431 5.897339820861816
40000 / 9892431 5.770260334014893
50000 / 9892431 5.3796305656433105
60000 / 9892431 5.561599254608154
70000 / 9892431 5.349597454071045
80000 / 9892431 5.167945384979248
90000 / 9892431 5.255443572998047
100000 / 9892431 4.909168720245361
110000 / 9892431 5.204761505126953
120000 / 9892431 4.7988667488098145
130000 / 9892431 4.9238810539245605
140000 / 9892431 4.994549751281738
150000 / 9892431 5.09640645980835
160000 / 9892431 4.871365070343018
170000 / 9892431 4.951051712036133
180000 / 9892431 4.733196258544922
190000 / 9892431 4.6749162673950195
200000 / 9892431 4.84122896194458
210000 / 9892431 4.635652542114258
220000 / 9892431 4.629925727844238
230000 / 9892431 4.667011737823486
240000 / 9892431 4.581827163696289
250000 / 9892431 4.516125679016113
260000 / 9892431 4.6436285972595215
270000 / 9892431 4.523196697235107
280000 / 9892431 4.66642475128

In [10]:
del model

##   Вариант с използване на последователност от модули

In [11]:
model = torch.nn.Sequential(
                            torch.nn.Embedding(L, emb_size),
                            torch.nn.Flatten(1,2),
                            torch.nn.Linear((order-1) * emb_size, hid_size),
                            torch.nn.Sigmoid(),
                            torch.nn.Linear(hid_size, L)
                            ).to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=1e-2, weight_decay=0.000001)

for b in range(0,len(idx),batchSize):
    
    batchIdx = idx[b:min(b+batchSize,len(idx))]
    batchTarget = torch.tensor(target[batchIdx], dtype=torch.long, device = device)
    batchContext = torch.tensor(context[batchIdx], dtype=torch.long, device = device)
    
    z = model(batchContext)
    H = torch.nn.functional.cross_entropy(z,batchTarget)
    
    optimizer.zero_grad()
    H.backward()
    optimizer.step()

    if b % 10000 == 0:
        print(b, '/', len(idx), H.item())


0 / 9892431 9.94266414642334
10000 / 9892431 6.4941182136535645
20000 / 9892431 6.266105651855469
30000 / 9892431 5.889111518859863
40000 / 9892431 5.740289211273193
50000 / 9892431 5.3468756675720215
60000 / 9892431 5.5436482429504395
70000 / 9892431 5.339859485626221
80000 / 9892431 5.144783973693848
90000 / 9892431 5.233212947845459
100000 / 9892431 4.882155418395996
110000 / 9892431 5.186201095581055
120000 / 9892431 4.745685577392578
130000 / 9892431 4.924731254577637
140000 / 9892431 4.981329917907715
150000 / 9892431 5.091183185577393
160000 / 9892431 4.8712615966796875
170000 / 9892431 4.9456095695495605
180000 / 9892431 4.7355852127075195
190000 / 9892431 4.669179916381836
200000 / 9892431 4.829428195953369
210000 / 9892431 4.603466033935547
220000 / 9892431 4.618254661560059
230000 / 9892431 4.662724494934082
240000 / 9892431 4.589049339294434
250000 / 9892431 4.5175395011901855
260000 / 9892431 4.65531063079834
270000 / 9892431 4.555191516876221
280000 / 9892431 4.6419172286

In [14]:
def perplexity(model, testCorpus, word2ind, order, device, batchSize):
    target, context = extractData(testCorpus, order, word2ind)
    
    with torch.no_grad():
        H = 0.
        for b in range(0,len(target),batchSize):
            batchTarget = torch.tensor(target[b:min(b+batchSize,len(target))], dtype=torch.long, device = device)
            batchContext = torch.tensor(context[b:min(b+batchSize,len(target))], dtype=torch.long, device = device)
            l = len(batchTarget)
            
            z = model(batchContext)
            H += l * torch.nn.functional.cross_entropy(z,batchTarget)
            del batchTarget
            del batchContext
            del l
            del z

    return math.exp(H/len(target))

In [23]:
torch.cuda.empty_cache()

In [38]:
print(len(testCorpus))
print(len(trainCorpus))

3922
388284


In [16]:
print(perplexity(model, testCorpus, word2ind, order, device, batchSize))

[--------------------------------------------------]
38.628727258793276


In [28]:
print(torch.cuda.memory_allocated(device) / 1024 / 1024 / 1024)
print(torch.cuda.memory_reserved(device) / 1024 / 1024 / 1024)

6.370616436004639
6.47265625


## Разбиване на корпуса на партиди с изречения с еднаква дължина

In [41]:
def splitCorpusInBatches(corpus, batchSize):
    minLen = min(len(s) for s in corpus)
    maxLen = max(len(s) for s in corpus)
    
    corpusBins = [ []  for _ in range(maxLen - minLen + 1) ]
    for s in corpus:
        l = len(s) - minLen
        corpusBins[l].append(s)
    
    batchCorpus = []
    for l in range(maxLen - minLen + 1):
        bin = corpusBins[l]
        idx = np.arange(len(bin), dtype='int32')
        np.random.shuffle(idx)
        for b in range(0, len(bin), batchSize):
            batch = []
            for si in range(b, min(b + batchSize, len(bin))):
                batch.append(bin[idx[si]])
            batchCorpus.append(batch)
    return batchCorpus

## LSTM

In [40]:
class LSTMCellModel(torch.nn.Module):
    def __init__(self, embed_size, hidden_size):
        super(LSTMCellModel, self).__init__()
        self.ii = torch.nn.Linear(embed_size, hidden_size)
        self.fi = torch.nn.Linear(embed_size, hidden_size)
        self.oi = torch.nn.Linear(embed_size, hidden_size)
        self.gi = torch.nn.Linear(embed_size, hidden_size)
        self.ih = torch.nn.Linear(hidden_size, hidden_size)
        self.fh = torch.nn.Linear(hidden_size, hidden_size)
        self.oh = torch.nn.Linear(hidden_size, hidden_size)
        self.gh = torch.nn.Linear(hidden_size, hidden_size)
    
    def forward(self, input, hc_0):
        (h_0, c_0) = hc_0
        i = torch.sigmoid(self.ii(input) + self.ih(h_0))
        f = torch.sigmoid(self.fi(input) + self.fh(h_0))
        o = torch.sigmoid(self.oi(input) + self.oh(h_0))
        g = torch.tanh(self.gi(input) + self.gh(h_0))
        c_1 = f * c_0 + i * g
        h_1 = o * torch.tanh(c_1)
        return (h_1, c_1)

class LSTMModel(torch.nn.Module):
    def __init__(self, embed_size, hidden_size):
        super(LSTMModel, self).__init__()
        self.hidden_size = hidden_size
        #self.cell = LSTMCellModel(embed_size, hidden_size)
        self.cell = torch.nn.LSTMCell(embed_size, hidden_size)
    
    def forward(self, input):
    	# input.shape = ( seq_len, batch_size, embed_size)
        seq_len = input.shape[0]
        batch_size = input.shape[1]
        device = next(self.parameters()).device
        h = torch.zeros(batch_size,self.hidden_size, device = device)
        c = torch.zeros(batch_size,self.hidden_size, device = device)
        output = []
        for i in range(seq_len):
            h, c = self.cell(input[i], (h, c))
            output.append(h)
        return torch.stack(output), (h,c)

class LSTMLanguageModel(torch.nn.Module):
    def __init__(self, embed_size, hidden_size, word2ind, unkToken):
        super(LSTMLanguageModel, self).__init__()
        self.word2ind = word2ind
        self.unkTokenIdx = word2ind[unkToken]
        #self.lstm = LSTMModel(embed_size, hidden_size)
        self.lstm = torch.nn.LSTM(embed_size, hidden_size)
        self.embed = torch.nn.Embedding(len(word2ind), embed_size)
        self.projection = torch.nn.Linear(hidden_size,len(word2ind))
    
    def forward(self, source):
        ### source e списък от изречения с равна дължина. Всяко изречение е списък от думи
        device = next(self.parameters()).device
        batch_size = len(source)
        sents = [[self.word2ind.get(w,self.unkTokenIdx) for w in s] for s in source]
        X = torch.t(torch.tensor(sents, dtype=torch.long, device=device))
        # X.shape = ( seq_len, batch_size )
        E = self.embed(X[:-1])
        output, _ = self.lstm(E)
        # output.shape = seq_len, batch_size, hidden_size )
        Z = self.projection(output.flatten(0,1))
        Y_bar = X[1:].flatten(0,1)
        H = torch.nn.functional.cross_entropy(Z,Y_bar)
        return H


In [54]:
batchSize = 32
batchCorpus = splitCorpusInBatches(trainCorpus, batchSize)

lm = LSTMLanguageModel(emb_size, hid_size, word2ind, unkToken).to(device)
optimizer = torch.optim.Adam(lm.parameters(), lr=0.01)

idx = np.arange(len(batchCorpus), dtype='int32')
np.random.shuffle(idx)


In [27]:
del lm

In [55]:
# train
for b in range(len(idx)):
    H = lm(batchCorpus[idx[b]])
    optimizer.zero_grad()
    H.backward()
    optimizer.step()
    if b % 10 == 0:
        print(b, '/', len(idx), H.item())

0 / 12228 9.896366119384766
10 / 12228 6.422996520996094
20 / 12228 6.579193115234375
30 / 12228 5.986417770385742
40 / 12228 5.989391326904297
50 / 12228 5.738386154174805
60 / 12228 5.81785774230957
70 / 12228 5.3573431968688965
80 / 12228 5.65693998336792
90 / 12228 5.597925662994385
100 / 12228 5.572377681732178
110 / 12228 5.221465110778809
120 / 12228 4.977700233459473
130 / 12228 5.333959579467773
140 / 12228 5.14569091796875
150 / 12228 5.224305152893066
160 / 12228 5.199784278869629
170 / 12228 5.275382041931152
180 / 12228 4.365095138549805
190 / 12228 5.053351879119873
200 / 12228 5.036250591278076
210 / 12228 4.979861736297607
220 / 12228 5.113883018493652
230 / 12228 4.589376449584961
240 / 12228 4.608710289001465
250 / 12228 4.038299083709717
260 / 12228 4.579703330993652
270 / 12228 4.475098609924316
280 / 12228 4.5625762939453125
290 / 12228 4.150399684906006
300 / 12228 4.754954814910889
310 / 12228 4.696532249450684
320 / 12228 4.135110855102539
330 / 12228 4.68404436

In [51]:
def perplexity(lm, testCorpus, batchSize):
    batchCorpus = splitCorpusInBatches(testCorpus, batchSize)
    H = 0.
    c = 0
    for b in range(len(batchCorpus)):
        l = len(batchCorpus[b])*(len(batchCorpus[b][0])-1)
        c += l
        with torch.no_grad():
            H += l * lm(batchCorpus[b])
    return math.exp(H/c)


In [57]:
print(perplexity(lm, testCorpus, batchSize))


35.97475045243066


## LSTM с пакетиране на партида

In [18]:
class LSTMLanguageModelPack(torch.nn.Module):
    def __init__(self, embed_size, hidden_size, word2ind, unkToken, padToken):
        super(LSTMLanguageModelPack, self).__init__()
        self.word2ind = word2ind
        self.unkTokenIdx = word2ind[unkToken]
        self.padTokenIdx = word2ind[padToken]
        self.lstm = torch.nn.LSTM(embed_size, hidden_size)
        self.embed = torch.nn.Embedding(len(word2ind), embed_size)
        self.projection = torch.nn.Linear(hidden_size,len(word2ind))
    
    def preparePaddedBatch(self, source):
        device = next(self.parameters()).device
        m = max(len(s) for s in source)
        sents = [[self.word2ind.get(w,self.unkTokenIdx) for w in s] for s in source]
        sents_padded = [ s+(m-len(s))*[self.padTokenIdx] for s in sents]
        return torch.t(torch.tensor(sents_padded, dtype=torch.long, device=device))

    def forward(self, source):
        X = self.preparePaddedBatch(source)
        E = self.embed(X[:-1])
        source_lengths = [len(s)-1 for s in source]
        outputPacked, _ = self.lstm(torch.nn.utils.rnn.pack_padded_sequence(E, source_lengths,enforce_sorted=False))
        output,_ = torch.nn.utils.rnn.pad_packed_sequence(outputPacked)

        Z = self.projection(output.flatten(0,1))
        Y_bar = X[1:].flatten(0,1)
        H = torch.nn.functional.cross_entropy(Z,Y_bar,ignore_index=self.padTokenIdx)
        return H


In [19]:
padToken = '<pad>'
def extractDictionary(corpus, limit=20000):
    pb = progressBar()
    pb.start(len(corpus))
    dictionary = {}
    for doc in corpus:
        pb.tick()
        for w in doc:
            if w not in dictionary: dictionary[w] = 0
        dictionary[w] += 1
    L = sorted([(w,dictionary[w]) for w in dictionary], key = lambda x: x[1] , reverse=True)
    if limit > len(L): limit = len(L)
    words = [ w for w,_ in L[:limit] ] + [unkToken] + [padToken]
    word2ind = { w:i for i,w in enumerate(words)}
    pb.stop()
    return words, word2ind

words, word2ind = extractDictionary(corpus)



[                                                  ]

[--------------------------------------------------]


In [20]:
lm = LSTMLanguageModelPack(emb_size, hid_size, word2ind, unkToken, padToken).to(device)
optimizer = torch.optim.Adam(lm.parameters(), lr=0.01)

idx = np.arange(len(trainCorpus), dtype='int32')
np.random.shuffle(idx)

In [21]:
#train
for b in range(0, len(idx), batchSize):
    batch = [ trainCorpus[i] for i in idx[b:min(b+batchSize, len(idx))] ]
    H = lm(batch)
    optimizer.zero_grad()
    H.backward()
    optimizer.step()
    if b % 10 == 0:
        print(b, '/', len(idx), H.item())

OutOfMemoryError: HIP out of memory. Tried to allocate 6.11 GiB. GPU 0 has a total capacity of 7.98 GiB of which 1.11 GiB is free. Of the allocated memory 6.37 GiB is allocated by PyTorch, and 104.49 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)

In [72]:
def perplexity(lm, testCorpus, batchSize):
    H = 0.
    c = 0
    for b in range(0,len(testCorpus),batchSize):
        batch = testCorpus[b:min(b+batchSize, len(testCorpus))]
        l = sum(len(s)-1 for s in batch)
        c += l
        with torch.no_grad():
            H += l * lm(batch)
    return math.exp(H/c)

In [73]:
print(perplexity(lm, testCorpus, batchSize))

68.92782089659758


In [68]:
torch.cuda.empty_cache()